In [1]:
import findspark
findspark.init()
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pyspark as ps
spark = SparkSession.builder \
    .master('local[4]') \
    .appName('project#1') \
    .getOrCreate()

sc = spark.sparkContext
spark = ps.sql.SQLContext(sc)

In [2]:
import pandas as pd
import numpy as np

In [3]:
datas=(sc.textFile("mydata.csv") 
    .map(lambda line: line.split(","))
    .filter(lambda x: x[1] in ['RCB','SRH','MI','RPS','GL','KKR','KXIP','DD'] )  
    .map(lambda x:(x[2],int(x[4])))
    .groupByKey()
    .map(lambda x:(x[0],np.max(list(x[1])),np.min(list(x[1])),np.quantile(list(x[1]),0.25),np.quantile(list(x[1]),0.5),np.quantile(list(x[1]),0.75)))
    .collect())

In [4]:
my_df=pd.DataFrame(datas,columns=['Player','Max_Score','Min_score','Frst_quantile','Median','Last_quantile'])

my_df caculates performance of all player in every team  as asked in Question 2

In [5]:
my_df.head()

,Player,Max_Score,Min_score,Frst_quantile,Median,Last_quantile
0,CH Gayle,77,0,6.00,8.0,32.00
1,Sachin Baby,12,1,1.50,2.0,7.00
2,TS Mills,6,0,1.00,2.0,4.00
3,DJ Hooda,19,9,9.75,12.5,15.25
4,Yuvraj Singh,70,0,5.50,9.0,36.50


In [6]:
def fn(A):
    k=A[0]
    B=A[1]
    cluster=[]
    for i in range(0,len(B),3):
        cluster.append(B[i:i+3])
    teams=cluster[0][0:2]
    team_1=filter(lambda x:x[0]==teams[0],cluster)
    team_2=filter(lambda x:x[0]==teams[1],cluster)
    s=0
    for c in team_1:
        s=s+c[2]
    u=0
    for c in team_2:
        u=u+c[2]
    p=1
    w=0
    if s>u:
        w=1
    return k,teams[0],teams[1],s,u,p,w

In [7]:
def f1(A):
    team_1=A[2]
    team_2=A[1]
    s=A[4]
    u=A[3]
    p=1
    w=0
    if s>u:
        w=1
    return (A[0],team_1,team_2,s,u,p,w)

In [8]:
def f2(A):
    k=A[0]
    B=A[1]
    cluster=[]
    dict_1={}
    for i in range(0,len(B),2):
        cluster.append(B[i:i+2])
    for i in cluster:
        if i[0] in dict_1.keys():
            dict_1[i[0]]=dict_1[i[0]]+i[1]
        else:
            dict_1[i[0]]=i[1]
    values=[]
    for i in dict_1.keys():
        values.append(2*dict_1[i])
    return (k,values)

In [9]:
from operator import add
data2=(sc.textFile("mydata.csv") 
    .map(lambda line: line.split(","))
    .filter(lambda x: x[1] in ['RCB','SRH','MI','RPS','GL','KKR','KXIP','DD'] )
    .map(lambda x:(int(x[0]),(x[1],x[3],int(x[5]))))
    .reduceByKey(add) 
    .sortByKey()  
    .map(fn)
    .flatMap(lambda x:(x,f1(x)))
    .map(lambda x:(x[1],(x[2],x[6]))) 
    .reduceByKey(add) 
    .map(f2)
    .map(lambda x:(x[0],np.max(list(x[1])),np.min(list(x[1])),np.quantile(list(x[1]),0.25),np.quantile(list(x[1]),0.5),np.quantile(list(x[1]),0.75)))  
    .collect())
# print(data2)

In [10]:
df=pd.DataFrame(data2,columns=['Team','Max_Score','Min_score','Frst_quantile','Median','Last_quantile'])

The dataframe my_df calculates the boxplot metrics of all team overall performances based on player performances as asked in Question 3.

In [11]:
df

,Team,Max_Score,Min_score,Frst_quantile,Median,Last_quantile
0,RCB,4,0,0.0,0.0,1.0
1,SRH,4,0,2.0,2.0,4.0
2,MI,6,2,2.0,4.0,4.0
3,RPS,6,0,2.0,2.0,4.0
4,KKR,4,0,2.0,2.0,3.0
5,GL,2,0,0.0,2.0,2.0
6,KXIP,4,0,2.0,2.0,2.0
7,DD,4,0,0.0,2.0,3.0


In [12]:
def f3(A):
    k=A[0]
    B=A[1]
    cluster=[]
    dict_1={}
    for i in range(0,len(B),2):
        cluster.append(B[i:i+2])
    for i in cluster:
        if i[0] in dict_1.keys():
            dict_1[i[0]]=dict_1[i[0]]+i[1]
        else:
            dict_1[i[0]]=i[1]
    values=[]
    for i in dict_1.keys():
        values.append((k,i,dict_1[i]))
        
    return values

In [13]:
dat=(sc.textFile("mydata.csv") 
    .map(lambda line: line.split(","))
    .filter(lambda x: x[1] in ['RCB','SRH','MI','RPS','GL','KKR','KXIP','DD'] )
    .map(lambda x:(int(x[0]),(x[1],x[3],int(x[5]))))
    .reduceByKey(add) 
    .sortByKey()  
    .map(fn)
    .flatMap(lambda x:(x,f1(x)))
    .map(lambda x:(x[1],(x[2],x[6]))) 
    .reduceByKey(add) 
    .flatMap(f3)
    .collect())
# print(dat)

In [14]:
dat_1=(sc.textFile("mydata.csv") 
    .map(lambda line: line.split(","))
    .filter(lambda x: x[1] in ['RCB','SRH','MI','RPS','GL','KKR','KXIP','DD'] )
    .map(lambda x:(int(x[0]),(x[1],x[3],int(x[5]))))
    .reduceByKey(add) 
    .sortByKey()  
    .map(fn)
    .flatMap(lambda x:(x,f1(x)))
    .map(lambda x:(x[1],(x[2],x[5]))) 
    .reduceByKey(add) 
    .flatMap(f3)
    .collect())
# print(dat_1)

In [15]:
vals=[]
for i in range(len(dat)):
    A=dat[i]
    B=dat_1[i]
    c=float(A[-1])/float(B[-1])
    vals.append((A[0],A[1],c))
# print(vals)

In [16]:
df_like=pd.DataFrame(vals,columns=['team_1','team_2','probability_win_team_1over_team_2'])

df_like represents the probabilty  that team_1 wins over team_2 as asked in Question 4

In [17]:
print(df_like)

   team_1 team_2  probability_win_team_1over_team_2
0     RCB    SRH                           0.000000
1     RCB     DD                           1.000000
2     RCB   KXIP                           0.000000
3     RCB     MI                           0.000000
4     RCB    RPS                           0.000000
5     RCB     GL                           0.500000
6     RCB    KKR                           0.000000
7     SRH    RCB                           1.000000
8     SRH     GL                           1.000000
9     SRH     MI                           0.500000
10    SRH    KKR                           0.666667
11    SRH   KXIP                           1.000000
12    SRH     DD                           0.500000
13    SRH    RPS                           0.000000
14     MI    RPS                           0.250000
15     MI    KKR                           1.000000
16     MI    SRH                           0.500000
17     MI    RCB                           1.000000
18     MI   

In [18]:
def rank(A):
    ranks={'RCB':8,'SRH':3,'MI':1,'RPS':2,'GL':7,'KKR':4,'KXIP':5,'DD':6}
    return A[0],A[1],A[2],ranks[A[0]]

In [19]:
from operator import add
data1=(sc.textFile("mydata.csv") 
    .map(lambda line: line.split(","))
    .filter(lambda x: x[1] in ['RCB','SRH','MI','RPS','GL','KKR','KXIP','DD'] )
    .map(lambda x:(int(x[0]),(x[1],x[3],int(x[5]))))
    .reduceByKey(add) 
    .sortByKey()  
    .map(fn)
    .flatMap(lambda x:(x,f1(x)))
    .map(lambda x:(x[1],(x[2],x[5]))) 
    .reduceByKey(add) 
    .flatMap(f3)
    .map(rank))
df=spark.createDataFrame(data1,['Team_1','Team_2','n_mathes_played','labels'])

In [20]:
df.show(10)

+------+------+---------------+------+
|Team_1|Team_2|n_mathes_played|labels|
+------+------+---------------+------+
|   RCB|   SRH|              1|     8|
|   RCB|    DD|              2|     8|
|   RCB|  KXIP|              2|     8|
|   RCB|    MI|              2|     8|
|   RCB|   RPS|              2|     8|
|   RCB|    GL|              2|     8|
|   RCB|   KKR|              2|     8|
|   SRH|   RCB|              1|     3|
|   SRH|    GL|              2|     3|
|   SRH|    MI|              2|     3|
+------+------+---------------+------+
only showing top 10 rows



In [21]:
new_data=(sc.textFile("random.txt") 
    .map(lambda line: line.split("\t"))
    .map(lambda x:(x[1],x[2],int(x[3])))
    .filter(lambda x:x[2]!=0)
    .collect())
df_predict_1=spark.createDataFrame(new_data,['Team_1','Team_2','n_mathes_played'])
# print(new_data)

In [22]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorAssembler,OneHotEncoder

In [23]:
cat_cols=df.columns[0:-1]

In [24]:
stringindexer_stages = [StringIndexer(inputCol=c, outputCol='strindexed_' + c) for c in cat_cols]

In [25]:
onehotencoder_stages = [OneHotEncoder(inputCol='strindexed_' + c, outputCol='onehot_' + c) for c in cat_cols]

In [26]:
feature_columns = ['onehot_' + c for c in cat_cols]
vectorassembler_stage = VectorAssembler(inputCols=feature_columns, outputCol='features')

In [27]:
all_stages = stringindexer_stages + onehotencoder_stages + [vectorassembler_stage]
pipeline = Pipeline(stages=all_stages)

In [28]:
pipeline_model = pipeline.fit(df)

In [29]:
final_columns = feature_columns + ['Team_1','features', 'labels']
new_df = pipeline_model.transform(df).\
            select(final_columns)
# new_df.show(5)

In [30]:
dt = DecisionTreeClassifier(featuresCol='features', labelCol='labels')

In [31]:
cv_model = dt.fit(new_df)

In [32]:
final_columns = feature_columns + ['Team_1','features']
df_pred_1 = pipeline_model.transform(df_predict_1).\
            select(final_columns)

Ranks of team in the new season as asked in Question 5

In [33]:
show_columns = ['Team_1', 'prediction']
pred_training_cv = cv_model.transform(df_pred_1)
pred_training_cv.select(show_columns).show(8, truncate=False)

+------+----------+
|Team_1|prediction|
+------+----------+
|SRH   |2.0       |
|MI    |1.0       |
|RPS   |2.0       |
|GL    |7.0       |
|KKR   |4.0       |
|KXIP  |5.0       |
|DD    |6.0       |
|RCB   |2.0       |
+------+----------+
only showing top 8 rows



In [34]:
def win(A):
    w=0
    if A[0]=='MI':
        w=1
    return A[0],A[1],A[2],w

In [35]:
d1=(sc.textFile("mydata.csv") 
    .map(lambda line: line.split(","))
    .filter(lambda x: x[1] in ['RCB','SRH','MI','RPS','GL','KKR','KXIP','DD'] )
    .map(lambda x:(int(x[0]),(x[1],x[3],int(x[5]))))
    .reduceByKey(add) 
    .sortByKey()  
    .map(fn)
    .flatMap(lambda x:(x,f1(x)))
    .map(lambda x:(x[1],(x[2],x[5]))) 
    .reduceByKey(add) 
    .flatMap(f3)
    .map(win)
   .collect())
d1_df=spark.createDataFrame(d1,['Team_1','Team_2','n_mathes_played','labels'])
# print(d1)

In [36]:
cat_cols=d1_df.columns[0:-1]

In [37]:
stringindexer_stages = [StringIndexer(inputCol=c, outputCol='strindexed_' + c) for c in cat_cols]

In [38]:
onehotencoder_stages = [OneHotEncoder(inputCol='strindexed_' + c, outputCol='onehot_' + c) for c in cat_cols]

In [39]:
feature_columns = ['onehot_' + c for c in cat_cols]
vectorassembler_stage = VectorAssembler(inputCols=feature_columns, outputCol='features')

In [40]:
all_stages = stringindexer_stages + onehotencoder_stages + [vectorassembler_stage]
pipeline = Pipeline(stages=all_stages)

In [41]:
pipeline_model_1 = pipeline.fit(d1_df)

In [42]:
final_columns = feature_columns + ['Team_1','features', 'labels']
new_d1_df = pipeline_model_1.transform(d1_df).\
            select(final_columns)
# new_d1_df.show(5)

In [43]:
from pyspark.ml.classification import LogisticRegression
logr = LogisticRegression(featuresCol='features', labelCol='labels')

In [44]:
lr_model = logr.fit(new_d1_df)

In [45]:
final_columns_1 = feature_columns + ['Team_1','features']
df_pred_1 = pipeline_model_1.transform(df_predict_1).\
            select(final_columns_1)

In [88]:
# df_pred_1.show(5)

In [46]:
show_columns = ['Team_1', 'prediction','probability']
pred_training_lr = lr_model.transform(df_pred_1)
pred_training_lr.select(show_columns).show(8, truncate=False)

+------+----------+------------------------------------------+
|Team_1|prediction|probability                               |
+------+----------+------------------------------------------+
|SRH   |0.0       |[0.9999999287932635,7.120673665259581E-8] |
|MI    |1.0       |[4.436711765646851E-9,0.9999999955632881] |
|RPS   |0.0       |[0.9999963494021604,3.6505978394815447E-6]|
|GL    |0.0       |[0.9999999976848515,2.3151485392682334E-9]|
|KKR   |0.0       |[0.9999669570219757,3.3042978024386555E-5]|
|KXIP  |0.0       |[0.9999999944901519,5.509848051069725E-9] |
|DD    |0.0       |[0.9999999976848515,2.3151485392682334E-9]|
|RCB   |0.0       |[0.9999999820722162,1.7927783900410936E-8]|
+------+----------+------------------------------------------+
only showing top 8 rows



In [47]:
de=pred_training_lr.select(show_columns).toPandas()

In [48]:
de['prob']=de['probability'].apply(lambda x:x[1])

In [49]:
de=de.drop(['prediction','probability'],axis=1)

Probability of a team winning the new season as asked in Question 6

In [50]:
de.groupby('Team_1').mean()

,prob
Team_1,
DD,3.678512e-06
GL,3.568309e-05
KKR,4.755662e-06
KXIP,3.310891e-05
MI,1.000000e+00
RCB,5.274940e-06
RPS,7.357424e-07
SRH,2.219973e-04
